## Textify - Text to Spotify playlist

We start by importing the libraries.

In [ ]:
import pandas as pd
import numpy as n
import collections 
import spotipy
import spotipy.util as util
import string
import regex as re
import time 

##### Spotify Access Token

You need to have a client ID and a client secret key in order to use Spotify API.

You can get these credentials from [Spotify Developers website](https://developer.spotify.com/dashboard/). You need to create an account if you don't have one.

In [ ]:
#Put CLIENT ID
CLIENT_ID = '####################' 
#Put CLIENT SECRET
CLIENT_SECRET = '#########################'

#Get Access Token
token = util.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

##### Used Functions

We use this function to remove any type of punctuation from the entered text.

In [266]:
def remove_punctuation(text):
    return re.sub(ur"\p{P}+", "", text)

This function returns the result tracks of a search with a limit set to 30 results. You can find more information about the Spotify API search function [here](https://developer.spotify.com/documentation/web-api/reference/search/search/).

In [267]:
def search_song(track):
    return sp.search(q='track:'+track, type='track',limit=30)

This function retrieves the song name, artist name and the song spotify URI from the search results and returns them in a list.

In [268]:
def get_result_songs(text):
    result_search=search_song(text)['tracks']['items']
    data=[]
    if len(result_search)==0:
        return data

    for result in result_search:
        entry={}
        entry['name'] = result['name']
        entry['artist']  = result['artists'][0]['name']
        entry['uri']= result['uri']
        data.append(entry)
    
    return data

The next function gets the whole text and follows these steps to transform it into a playlist: 
- Transform the text to a lower format and remove the punctuation
- Split the text and put it in a stack
- Loop through the list of words in the stack
- Each loop, we get the first 5 elements of the stack
- We create a loop with the stack of 5 elements, each time we joine them in a string, then perform a spotify search to try to find a match with a song. If we cannot find a match we pop the last element and try to find a match with the 4 elements and so on. The loop ends until there's no word in the stack of 5 elemnts or when there's a match.
- If there's a match we then increase our pointer by the number of elements used to get a match with a song, if not we increase it by 1 and just skip the word.

###### Example

entered text : "Hello how are you today"

First Loop : 

- 1st test : look for "hello how are you today" but no match.
- 2nd test : look for "hello how are you" but no match.
- 3rd test : look for "hello how are" but no match.
- 4th test : look for "hello how" but no match.
- 5th test : look for "hello" and we match with Adele - Hello.

Second Loop :

- 1st test : look for "how are you today" and we match with Christopher Rau - How Are You Today ?

The problem I encountered is that I couldn't find a way to force exact matching in Spotify queries and sometimes it limits the quality of the returned results compared with the search query. :(

In [269]:
def get_songs(text_entry):
    text_entry=remove_punctuation(text_entry.lower())
    text_entry=text_entry.split()
    i=0
    n=len(text_entry)
    songs=[]
    while i<n:
        split=text_entry[i:i+5]
        tag_found=False
        while len(split)>0 and not tag_found:
            text=' AND '.join(split)
            text_unsplitted=' '.join(split)
            get_results=get_result_songs(text)
            if len(get_results)!=0:
                for element in get_results:
                    songname=remove_punctuation(element['name'])
                    songname=songname.lower()
                    if songname==text_unsplitted:
                        songs.append(element)
                        tag_found=True
                        break
            if tag_found:
                i=i+len(split)
            else:
                split.pop()

        if not tag_found:
            i=i+1
    
    return songs

### Test Examples

In [271]:
get_songs("Hello let's meet at the cinema tonigth")

[{'artist': u'Adele',
  'name': u'Hello',
  'uri': u'spotify:track:4sPmO7WMQUAf45kwMOtONw'},
 {'artist': u'Michelle Gurevich',
  'name': u"Let's Meet",
  'uri': u'spotify:track:16I9T2PXqy0nVpUPATNE0m'},
 {'artist': u'MeloMance',
  'name': u'At the Cinema',
  'uri': u'spotify:track:1fVdYz0JuL3p2petdaDGkP'},
 {'artist': u'Danny Groove',
  'name': u'Tonigth',
  'uri': u'spotify:track:4tB615lvL23bjRZLAMRP5U'}]

In [272]:
get_songs("The weather is really bad, I think I will stay at home and play with my cats.")

[{'artist': u'Lyves',
  'name': u'The Weather',
  'uri': u'spotify:track:3J7wh2tkjlGS2LmQmIkwYb'},
 {'artist': u'BLACKPINK',
  'name': u'REALLY',
  'uri': u'spotify:track:6OKXcx7tClGAS0o2cOTl2v'},
 {'artist': u'XXXTENTACION',
  'name': u'BAD!',
  'uri': u'spotify:track:4CH66Rxcjcj3VBHwmIBj4T'},
 {'artist': u'Faith Hill',
  'name': u'I Think I Will',
  'uri': u'spotify:track:4aqfREjGLoNTdSDS5qAJ2Y'},
 {'artist': u'Men At Work',
  'name': u'Stay at Home',
  'uri': u'spotify:track:14IfH2DuGCnm6OMkmYuzlP'},
 {'artist': u'Jax Jones',
  'name': u'Play',
  'uri': u'spotify:track:29R1IMTTbDDA3VNlk6UEW5'},
 {'artist': u'Macseal',
  'name': u'Cats',
  'uri': u'spotify:track:0Sz9cETbT6N4Pe33xvyHwC'}]

In [273]:
get_songs("Hello, how are you? Thank you. See you later.")

[{'artist': u'Dr. Jean Feldman',
  'name': u'Hello, How Are You?',
  'uri': u'spotify:track:4rTJmwy3HOPQQhqORzPqBD'},
 {'artist': u'Dido',
  'name': u'Thank You',
  'uri': u'spotify:track:751gBcu62kORDelX7FV0mM'},
 {'artist': u'Jaydayoungan',
  'name': u'See You Later',
  'uri': u'spotify:track:1mktWKQklKC1ImhBo4afnI'}]